# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
weather_df = pd.read_csv("cities.csv")

weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Port Alfred,1,ZA,1601328334,64,-33.59,26.89,56.80,10.49
1,1,Arraial do Cabo,66,BR,1601328488,94,-22.97,-42.03,75.20,8.05
2,2,Cherskiy,76,RU,1601328488,70,68.75,161.30,40.10,16.17
3,3,Mataura,100,NZ,1601328489,62,-46.19,168.86,37.99,20.00
4,4,Pevek,0,RU,1601328489,83,69.70,170.31,35.64,13.98
...,...,...,...,...,...,...,...,...,...,...
542,542,Spitsevka,15,RU,1601328712,58,45.12,42.51,57.20,13.42
543,543,Awasa,12,ET,1601328713,81,7.05,38.47,59.11,3.78
544,544,Sovkhoznyy,92,RU,1601328713,77,45.29,38.11,66.20,6.93
545,545,Sorong,100,ID,1601328713,84,-0.88,131.25,77.14,5.77


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]


In [26]:



# Create  Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
vacation_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

vacation_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
29,29,Buraidah,0,SA,1601328479,36,26.33,43.98,77.00,6.93
143,143,Mocuba,0,MZ,1601328540,57,-16.84,36.99,77.79,4.16
258,258,Jhelum,0,PK,1601328585,34,32.93,73.73,76.59,5.44
381,381,Medianeira,0,BR,1601328643,44,-25.30,-54.09,77.00,4.70
396,396,Gaur,0,NP,1601328651,82,26.77,85.27,78.28,2.86
465,465,Batāla,0,IN,1601328682,83,31.82,75.20,77.00,1.23
486,486,Marsá Maţrūḩ,0,EG,1601328690,83,31.35,27.25,78.80,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
# Create a hotel_df
hotel_df = vacation_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
29,Buraidah,SA,26.33,43.98,
143,Mocuba,MZ,-16.84,36.99,
258,Jhelum,PK,32.93,73.73,
381,Medianeira,BR,-25.30,-54.09,
396,Gaur,NP,26.77,85.27,
465,Batāla,IN,31.82,75.20,
486,Marsá Maţrūḩ,EG,31.35,27.25,


In [29]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [30]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # build url and make API call
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
     # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    

Retrieving Results for Index 29: Buraidah.
Closest hotel in Buraidah is Mövenpick Hotel Qassim.
Retrieving Results for Index 143: Mocuba.
Closest hotel in Mocuba is Pensão Venus.
Retrieving Results for Index 258: Jhelum.
Closest hotel in Jhelum is Tulip Riverside Hotel And Restaurant Jehlum.
Retrieving Results for Index 381: Medianeira.
Closest hotel in Medianeira is Hotel Passarela do Turismo.
Retrieving Results for Index 396: Gaur.
Closest hotel in Gaur is Royal Indu Hotel, Family Restaurant and Bar.
Retrieving Results for Index 465: Batāla.
Closest hotel in Batāla is Grandway-2 Hotel-Hotels in Batala.
Retrieving Results for Index 486: Marsá Maţrūḩ.
Closest hotel in Marsá Maţrūḩ is Beau Site.


In [31]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
29,Buraidah,SA,26.33,43.98,Mövenpick Hotel Qassim
143,Mocuba,MZ,-16.84,36.99,Pensão Venus
258,Jhelum,PK,32.93,73.73,Tulip Riverside Hotel And Restaurant Jehlum
381,Medianeira,BR,-25.30,-54.09,Hotel Passarela do Turismo
396,Gaur,NP,26.77,85.27,"Royal Indu Hotel, Family Restaurant and Bar"
465,Batāla,IN,31.82,75.20,Grandway-2 Hotel-Hotels in Batala
486,Marsá Maţrūḩ,EG,31.35,27.25,Beau Site


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info =[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
 

# Add marker layer and info box content on top of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Figure
fig



Figure(layout=FigureLayout(height='420px'))